In [244]:
# Import main librarires 
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 

In [245]:
# We are going to use Kaggle Datasets for this project 
# We need first to create an API Token so we can interact with the available datasets. Instructions are available on the Kaggle website
# Once API Token has been created we are importing the library in our notebook
import kaggle 

In [246]:
# Our project will focus on the relationship between Higher Educaiton, Employment Levels, Salaries, Health Status, and voting choice (democrats vs republicans)
# First we will start by downloading locally the data and reviewing it
kaggle.api.authenticate()
kaggle.api.dataset_download_files('satoshidatamoto/colleges-and-universities-a-comprehensive-datasee','/Users/petariliev/Downloads/Data ')

In [247]:
# Data is being donwloaded as a ZIP so we will need to unzip it first 
from zipfile import ZipFile
file_name = 'colleges-and-universities-a-comprehensive-datasee.zip'
with ZipFile(file_name, 'r') as zip: 
    zip.printdir() # Check all files in the zip 

File Name                                             Modified             Size
Colleges_and_Universities.csv                  2022-05-14 11:15:36      3665198


In [248]:
data_uni = pd.read_csv('Colleges_and_Universities.csv') #read the data as dataframe via pandas
data_uni.head() #check the top 5 records to review the data

,index,X,Y,FID,IPEDSID,NAME,ADDRESS,ADDRESS2,CITY,STATE,...,ALIAS,SIZE_SET,INST_SIZE,PT_ENROLL,FT_ENROLL,TOT_ENROLL,HOUSING,DORM_CAP,TOT_EMPLOY,SHELTER_ID
0,0,-92.260490,34.759308,7001,107840,Shorter College,604 Locust St,NOT AVAILABLE,N Little Rock,AR,...,NOT AVAILABLE,-3,1,24,28,52,2,0,18,NOT AVAILABLE
1,1,-121.289431,38.713353,7002,112181,Citrus Heights Beauty College,7518 Baird Way,NOT AVAILABLE,Citris Heights,CA,...,NOT AVAILABLE,-3,1,6,24,30,2,0,9,NOT AVAILABLE
2,2,-118.287070,34.101481,7003,116660,Joe Blasco Makeup Artist Training Center,1670 Hillhurst Avenue,NOT AVAILABLE,Los Angeles,CA,...,NOT AVAILABLE,-3,1,0,24,24,2,0,11,NOT AVAILABLE
3,3,-121.652662,36.700631,7004,125310,Waynes College of Beauty,1271 North Main Street,NOT AVAILABLE,Salinas,CA,...,NOT AVAILABLE,-3,1,18,16,34,2,0,9,NOT AVAILABLE
4,4,-71.070737,42.369930,7005,164368,Hult International Business School,1 Education Street,NOT AVAILABLE,Cambridge,MA,...,NOT AVAILABLE,-3,2,0,2243,2243,2,0,143,NOT AVAILABLE


In [249]:
print(data_uni.columns)

Index(['index', 'X', 'Y', 'FID', 'IPEDSID', 'NAME', 'ADDRESS', 'ADDRESS2',
       'CITY', 'STATE', 'ZIP', 'ZIP4', 'TELEPHONE', 'TYPE', 'STATUS',
       'POPULATION', 'COUNTY', 'COUNTYFIPS', 'COUNTRY', 'LATITUDE',
       'LONGITUDE', 'NAICS_CODE', 'NAICS_DESC', 'SOURCE', 'SOURCE_DAT',
       'VAL_METHOD', 'VAL_DATE', 'WEBSITE', 'STFIPS', 'COFIPS', 'SECTOR',
       'LEVEL_', 'HI_OFFER', 'DEG_GRANT', 'LOCALE', 'CLOSE_DATE', 'MERGE_ID',
       'ALIAS', 'SIZE_SET', 'INST_SIZE', 'PT_ENROLL', 'FT_ENROLL',
       'TOT_ENROLL', 'HOUSING', 'DORM_CAP', 'TOT_EMPLOY', 'SHELTER_ID'],
      dtype='object')


In [250]:
# Once we review all data we identify what columns we need for our task
# We will keep Name, Address, City, State, Zip, NAICS
# Although we have the NAICS_DESC we will create it manualy using the https://www.naics.com/naics-code-description/?code=611310

In [251]:
#creating the new DF 
data_edu = data_uni[['index', 'FID', 'NAME', 'ADDRESS', 'CITY', 'STATE', 'ZIP','COUNTY', 'NAICS_CODE']] 

In [252]:
data_edu.head()

,index,FID,NAME,ADDRESS,CITY,STATE,ZIP,COUNTY,NAICS_CODE
0,0,7001,Shorter College,604 Locust St,N Little Rock,AR,72114,Pulaski,611210
1,1,7002,Citrus Heights Beauty College,7518 Baird Way,Citris Heights,CA,95610,Sacramento,611511
2,2,7003,Joe Blasco Makeup Artist Training Center,1670 Hillhurst Avenue,Los Angeles,CA,90027,Los Angeles,611511
3,3,7004,Waynes College of Beauty,1271 North Main Street,Salinas,CA,93906,Monterey,611511
4,4,7005,Hult International Business School,1 Education Street,Cambridge,MA,02141,Middlesex,611310


In [253]:
data_edu['NAICS_CODE'].unique() #get all values for NAICS code to map 

array([611210, 611511, 611310, 611519, 611420, 611610, 611710, 611410,
       611512])

In [254]:
data_edu.dtypes  #check columns data types - will need further when joining and for creating the NAICS_CODE dict in the right data format

index          int64
FID            int64
NAME          object
ADDRESS       object
CITY          object
STATE         object
ZIP           object
COUNTY        object
NAICS_CODE     int64
dtype: object

In [255]:
#creating Dictionary for NAICS codes and converting to dataframe 
dict_naics = {'NAICS_CODE' : [611210, 611511, 611310, 611519, 611420, 611610, 611710 , 611410, 611512],
                  'NAICS_TYPE' : ['Junior Colleges', 'Cosmetology and Barber Schools', 'Colleges, Universities, and Professional Schools', 'Other Technical and Trade Schools', 'Computer Training', 'Fine Arts Schools',  'Educational Support Services', 'Business and Secretarial Schools', 'Flight Training']} 
                                  
df_dict_naics= pd.DataFrame.from_dict(dict_naics)   
df_dict_naics.head()

,NAICS_CODE,NAICS_TYPE
0,611210,Junior Colleges
1,611511,Cosmetology and Barber Schools
2,611310,"Colleges, Universities, and Professional Schools"
3,611519,Other Technical and Trade Schools
4,611420,Computer Training


In [256]:
df_dict_naics.dtypes

NAICS_CODE     int64
NAICS_TYPE    object
dtype: object

In [257]:
#joining the 2 dataframes based on column NAICS_CODE values 
data_edu = data_edu.join(df_dict_naics.set_index('NAICS_CODE'), on = 'NAICS_CODE')

In [258]:
data_edu.head()

,index,FID,NAME,ADDRESS,CITY,STATE,ZIP,COUNTY,NAICS_CODE,NAICS_TYPE
0,0,7001,Shorter College,604 Locust St,N Little Rock,AR,72114,Pulaski,611210,Junior Colleges
1,1,7002,Citrus Heights Beauty College,7518 Baird Way,Citris Heights,CA,95610,Sacramento,611511,Cosmetology and Barber Schools
2,2,7003,Joe Blasco Makeup Artist Training Center,1670 Hillhurst Avenue,Los Angeles,CA,90027,Los Angeles,611511,Cosmetology and Barber Schools
3,3,7004,Waynes College of Beauty,1271 North Main Street,Salinas,CA,93906,Monterey,611511,Cosmetology and Barber Schools
4,4,7005,Hult International Business School,1 Education Street,Cambridge,MA,02141,Middlesex,611310,"Colleges, Universities, and Professional Schools"


In [259]:
data_edu.drop(['index'], axis = 1) #we don't need index column from the dataset as dataframe has its own index

,FID,NAME,ADDRESS,CITY,STATE,ZIP,COUNTY,NAICS_CODE,NAICS_TYPE
0,7001,Shorter College,604 Locust St,N Little Rock,AR,72114,Pulaski,611210,Junior Colleges
1,7002,Citrus Heights Beauty College,7518 Baird Way,Citris Heights,CA,95610,Sacramento,611511,Cosmetology and Barber Schools
2,7003,Joe Blasco Makeup Artist Training Center,1670 Hillhurst Avenue,Los Angeles,CA,90027,Los Angeles,611511,Cosmetology and Barber Schools
3,7004,Waynes College of Beauty,1271 North Main Street,Salinas,CA,93906,Monterey,611511,Cosmetology and Barber Schools
4,7005,Hult International Business School,1 Education Street,Cambridge,MA,02141,Middlesex,611310,"Colleges, Universities, and Professional Schools"
...,...,...,...,...,...,...,...,...,...
7730,2996,Connecticut Center for Massage Therapy-Groton,1154 Poquonnock Rd,Groton,CT,06340,New London,611519,Other Technical and Trade Schools
7731,2997,Pierpont Community and Technical College,1201 Locust Ave,Fairmont,WV,26554,Marion,611210,Junior Colleges
7732,2998,Universal College of Beauty Inc-Compton,718 West Compton Blvd,Compton,CA,90220,Los Angeles,611710,Educational Support Services
7733,2999,ITT Technical Institute-Duluth,10700 Abbotts Bridge Rd-Ste 190,Duluth,GA,30097,Fulton,611310,"Colleges, Universities, and Professional Schools"


In [260]:
#we want to find data showing annual income per state and zip code

In [311]:
file_csv = '/Users/petariliev/Downloads/ACSST5Y2020.S1903_2022-05-15T093250/ACSST5Y2020.S1903_data_with_overlays_2022-04-26T163933.csv'
df_1 = pd.read_csv(file_csv, dtype = 'unicode')


In [312]:
#we need only the Geo Name area column and the median income for family household
data_zip = df_1[['NAME', 'S1903_C03_015E']]
data_zip.head()
#now we are dropping the first row of the table as we see data there is no useful but more like a sub header
data_zip = data_zip.iloc[1:, :]


In [313]:
#rename the column 
data_zip.rename (columns = {'S1903_C03_015E' : 'Median_Income_Household'}, inplace = True)

In [314]:
#reseting the index (due to droping the first row) and droping the index column 
data_zip.reset_index(inplace=True, drop=True)

In [315]:
data_zip.head()

,NAME,Median_Income_Household
0,ZCTA5 00601,15721
1,ZCTA5 00602,21060
2,ZCTA5 00603,19340
3,ZCTA5 00606,16176
4,ZCTA5 00610,23968


In [ ]:
#now we need to convert the ZCTA5 format to the ZIP Code format 
